In [1]:
import sys
sys.path.append("../")
import concurrent.futures
from tqdm import tqdm
import pandas as pd
import numpy as np
import openai
from io import StringIO
import re
import time
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
product_name = ["NIKI", "LE 5 A 7", "JAMIE", "ENVELOPE", "KATE", "MANHATTAN", "KATE TASSEL", "ICARE", "PUFFER", "SUNSET", "SOLFERINO", "LE 37", "LOULOU", "CASSANDRA", "LE 57", "KAIA", "CALYPSO", "SHOPPING SAINT LAURENT", "CASSANDRE MATELASSE", "NOLITA", "GABY", "SADE", "外套", "裙子", "T恤", "其他成衣", "男士成衣", "女士成衣", "高跟鞋", "其他鞋履", "皮带", "墨镜", "饰品"]
content_category = ["UGC", "PGC", "销售人员", "明星代言", "工艺与传承", "OOTD", "旅行体验", "女性力量", "好物推荐", "包包测评", "购物分享"]

In [3]:
system_prompt = f"""
    你是一个为 YSL 时尚社交聆听系统 服务的打标助手。你的任务是处理多条社交媒体帖子，并为每条帖子提取结构化标签。

⸻

以 CSV 格式 返回结果，例如：

    id,product_name,material,shape,bag_size,color,visual_element,functionality,style,emotion_expression,wearing_occation,content_category
"5","Niki","羊皮革","方包","小号","黑色","金色经典logo|品牌徽标磁扣开合","斜挎|单肩包","经典","太爱了|推荐","通勤|拍照|日常","种草分享"

以上只是一个例子，在实际标注过程中，如果信息缺失，请不要将其作为正确的信息使用。

    每条帖子由 id 标识。对于每条帖子，严格按照以下顺序提取 12 个字段：
	•	id：原始帖子 ID（原样复制）。
	•	product_name：提到的产品是什么，只应识别 YSL 时尚产品。一些名称示例如下：["{'","'.join(product_name)}”]。
	•	material：产品的主要材质（例如：油蜡皮、麂皮、羊皮、牛皮等）。
	•	shape：与产品造型相关的词语（例如：流浪包、托特包、水桶包、法棍包、托特包、猫眼镜、直筒连衣裙、修身上衣、喇叭裤等）。你可以结合对 YSL 产品的常识理解。
	•	bag_size：提及的包的尺寸（例如：迷你、小号、中号、24x18x6cm 等）。
	•	color：提及的产品颜色（例如：棕色、黑色、粉色等）。
	•	visual_element：标志性元素或印花（例如：豹纹、格纹、老花、YSL logo、印花等）。
	•	functionality：提及的突出产品功能（例如：大容量、可拆卸肩带、多种背法、有隔层、自重轻等）。
	•	style：美学风格（例如：优雅风、复古、cleanfit、大包回潮、Y2K、高级等）。同时考虑显式提及和隐含含义。例如：“草编”暗示“度假风”，“A4”暗示“大包回潮”，“无 logo”暗示“cleanfit”。
	•	emotion_expression：与提及产品的主观态度相关的词语（例如：气场全开、便宜、性价比等）。
	•	wearing_occation：使用场景（例如：通勤、度假、逛街、夏日、节日等）。同时考虑显式提及和隐含含义。例如：“可放电脑”暗示“通勤”，“打卡”暗示“拍照”。
	•	content_category：根据帖子内容推断，仅可从 ["{'","'.join(content_category)}”] 中选择。选择 2–3 个类别，当证据较弱时可以进行合理猜测。

⸻

重要规则：
	•	除了表头外，每条帖子仅对应一行。如果发现多款 YSL 产品，请将它们放在同一行。确保每行包含 12 个字段。
	•	如果发现多个关键词，用 | 分隔。不要在关键词中包含英文逗号或中文逗号，如有必要请将其拆分为多个关键词。
	•	只考虑与 YSL 时尚产品 相关的内容。忽略其他品牌和其他产品类别。
	•	帖子是中文的，如果关键词中出现表情符号，请将其转化为可理解的文字。例如：🉑、🈚️ 等。
	•	保持每个关键词简短且直接。尽可能多地提取关键词。
	•	对于缺失或不可用的值，用英文引号括起的 "no" 表示——不要留空。
	•	如果某条帖子中未发现任何 YSL 时尚产品，请跳过（不输出该行）。
	•	不要解释或总结——只输出 CSV 行。

    """

In [4]:
def is_strict_bad(df):
    # Expected columns
    expected_cols = [
        "id", "product_name", "material", "shape", "bag_size", "color", "visual_element", "functionality",
        "style", "emotion_expression", "wearing_occation", "content_category"]
    if list(df.columns) != expected_cols:
        return True

    # 1. Retry if empty table
    if df.empty:
        return True
    # 2. id must be all valid integers
    try:
        # This ensures no non-integer, NaN, or float
        if not df["id"].apply(lambda x: str(x).isdigit()).all():
            return True
    except Exception:
        return True

    return False

In [5]:
import time
from dashscope import Generation

# -------- 你的 API Key 直接写这里 --------
DASHSCOPE_API_KEY = "sk-c2bc15edd94a40d883cfb5f7362b6156"   # ⚠️ 注意替换为你自己的 key
# --------------------------------------

def Ask_GenAI(prompt, model="deepseek-v3", enable_thinking=False):
    """
    使用阿里云百炼 DashScope API 调用 DeepSeek 模型。
    输入 prompt，返回字符串输出内容。
    """
    # 组装 messages，与 OpenAI 格式相同
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
              ]
    
    # 调用 dashscope Generation 接口
    response = Generation.call(
        api_key=DASHSCOPE_API_KEY,
        model=model,
        messages=messages,
        result_format="message",      # 返回标准 message 格式
        enable_thinking=enable_thinking,  # 可选：是否启用思考模式
        stream=False,                 # 非流式，一次性拿完整输出
    )

    # 提取回复内容
    if hasattr(response, "output") and response.output and response.output.choices:
        content = response.output.choices[0].message.content
    else:
        content = "(No response)"
    
    time.sleep(1)  # 控制调用速率
    return content

In [6]:
# Format batched content for GPT
def make_user_message(df_batch):
    rows = []
    for _, row in df_batch.iterrows():
        id = row["index"]
        content = row["完整内容"]
        rows.append(f"id: {id}\ncontent: {content}\n")
    return "Tag the following posts one by one: \n\n" + "\n End of the post \n".join(rows)

def contains_invalid_keywords(text, invalid_list):
    if pd.isna(text):
        return False
    for kw in invalid_list:
        if kw in str(text).split("|"):
            return True
    return False

In [ ]:
cleaned_df = pd.read_csv("/root/ysl_project_clean/cleaned_df_v1.csv").iloc[:1000,]
cleaned_df_content = cleaned_df['合并内容']
cleaned_df_content.columns = ['combined content']

In [11]:
Ask_GenAI('HI')

'由于输入的文本 "HI" 不符合打标要求，且未提及任何 YSL 时尚产品，因此跳过处理该条帖子。'

In [ ]:
from io import StringIO
import pandas as pd
import re
import time

# ====== 可调参数 ======
N_SAMPLES   = 1000        # 只取前N条做验证；上线可改成 len(df)
BATCH_SIZE  = 8          # 每批多少行
# add 并行
RETRY_LIMIT = 1          # 每个批次失败后最多再重试几次
SLEEP_SEC   = 1          # 每个批次之间的停顿（防限流）

# ====== 取数据 ======
source_df = cleaned_df_content.iloc[:N_SAMPLES].copy().reset_index(drop=True)

# 期望的列顺序
expected_cols = [
    "id", "product_name", "material", "shape", "bag_size", "color",
    "visual_element", "functionality", "style", "emotion_expression",
    "wearing_occation", "content_category"
]

results = []
failed_logs = []

print(f"开始按批次处理数据... 共 {len(source_df)} 条，批大小 {BATCH_SIZE}\n")

# ====== 批次循环 ======
for start in range(0, len(source_df), BATCH_SIZE):
    end = min(start + BATCH_SIZE, len(source_df))
    batch_df = source_df.iloc[start:end]
    print(f"\n===== 🔹批次 {start}-{end-1}（共 {len(batch_df)} 条）=====")

    # 轻量重试
    ok = False
    for attempt in range(RETRY_LIMIT + 1):
        try:
            # 构造prompt（仍然用你现有的 make_user_message，直接喂入一个DataFrame批次）
            prompt = make_user_message(batch_df)

            # 调用大模型
            csv_output = Ask_GenAI(prompt)
            # 清除可能的 ```csv ... ``` 围栏
            csv_output = re.sub(r"^```csv\s*|\s*```$", "", csv_output.strip(), flags=re.MULTILINE)

            # 解析 CSV
            df_result = pd.read_csv(StringIO(csv_output))

            # 列对齐：缺哪些就补空列；然后按 expected_cols 排序
            for c in expected_cols:
                if c not in df_result.columns:
                    df_result[c] = pd.NA
            df_result = df_result[expected_cols]

            results.append(df_result)
            print(f"✅ 批次 {start}-{end-1} 成功（尝试 {attempt+1}/{RETRY_LIMIT+1}）")
            ok = True
            break

        except Exception as e:
            print(f"⚠️ 批次 {start}-{end-1} 第 {attempt+1} 次尝试报错：{e}")
            if attempt == RETRY_LIMIT:
                failed_logs.append({"batch_range": f"{start}-{end-1}", "reason": str(e)})

        # 失败则短暂停顿再试
        time.sleep(0.5)

    # 成功或已用尽重试后，做节流停顿
    time.sleep(SLEEP_SEC)

# ====== 汇总与导出 ======
print("\n✅ 处理完成。成功批次数：", len(results), " 失败批次数：", len(failed_logs))
if failed_logs:
    print("失败批次示例：", failed_logs[:3])

if results:
    content_tagging = pd.concat(results, ignore_index=True)
    content_tagging = content_tagging[expected_cols]  # 再次确保列顺序
    print(f"\n📁 已生成打标结果content_tagging")
else:
    print("❌ 无有效结果，未生成文件。")

In [ ]:
import pandas as pd

# 读取两个表
df = pd.read_csv("df.csv")  
content_tagging = pd.read_csv("ysl_tagging_batch_output.csv")

# 或者更稳妥一点的写法（推荐）
content_tagging = cleaned_df.reset_index().join(content_tagging.set_index('id'), how='inner')

# 导出结果
content_tagging.to_csv("content_tagging_v1.csv", index=False, encoding="utf-8-sig")

print(f"✅ 合并完成，共 {len(content_tagging)} 行，已保存为 content_tagging_v1. ")

# Quality assessment

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 假设 merged 已经加载
# merged = pd.read_csv("merged.csv")

# ===== 🧹 将 "no" 视为空值 =====
merged = merged_df.replace("no", np.nan)

# 要排除的列
exclude_cols = ['index', '完整内容']

# 筛选要分析的列
cols_to_check = [c for c in merged_df.columns if c not in exclude_cols]

# === 1️⃣ 空值比例与唯一值统计 ===
summary = []
for col in cols_to_check:
    null_ratio = merged_df[col].isna().mean()
    unique_count = merged_df[col].nunique(dropna=True)
    summary.append({
        'column': col,
        'null_ratio(%)': round(null_ratio * 100, 2),
        'unique_count': unique_count
    })

summary_df = pd.DataFrame(summary)
print("=== 🧾 Missing Value & Unique Summary ===")
print(summary_df.sort_values(by='null_ratio(%)', ascending=False))

=== 🧾 Missing Value & Unique Summary ===
                column  null_ratio(%)  unique_count
4             bag_size          79.06            64
2             material          77.32            86
6       visual_element          74.37           174
5                color          71.32            79
7        functionality          69.36           205
9   emotion_expression          53.87           346
10    wearing_occation          47.66           217
8                style          44.71           277
3                shape          40.57           269
1         product_name          26.72           107
11    content_category           6.98           155
0                 合并内容           0.00           914
